<a href="https://colab.research.google.com/github/Maplemx/Agently/blob/main/playground/constrast_between_Agently_workflow_and_LangGraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# The Constrast between Agently Workflow and LangGraph

**Author:** Agently Team

## Intro 简介

This document is a not-complete-compared introduction between Agently Workflow and LangGraph. We wrote this document due to many community developers wondering the differences between two framework modules. But because of the lack of time, we may have some lack of information or mistakes. If you discover those, please [report issues here](https://github.com/Maplemx/Agently/issues/new) or contact us, we would like to change this document after comfirming the mistakes to make this document objective.

本文档是Agently Workflow和LangGraph的不全面比较文档。因为很多社区开发者想要了解这两个框架模块之间的差异，我们撰写了这篇文档。但由于准备时间以及对LangGraph框架了解的深度限制，我们也可能在对比时有所纰漏和错误。如果您发现了这些问题，可以[点击这里向我们提交](https://github.com/Maplemx/Agently/issues/new)，或是用其他方式与我们去得联系。我们在确认问题后，会及时修改文档内容，来确保文档的客观性。

## Comparation Table 功能对比表

|Feature|Agently Workflow|LangGraph|
|---|---|---|
|**Create Workflow Instance**<br /><br />**创建工作流对象**|Created by `Agently.Workflow()`<br /><br />由`Agently.Workflow()`类创建|Created by `langgraph.graph.StateGraph()`<br />Customized structure class `State` is required for workflow data communication.<br /><br />通过`langgraph.graph.StateGraph()`类创建<br />需要传入自定义结构类`State`作为补充，用于后续工作流中的数据传递结构|
|**Define Workflow Chunk/Node**<br /><br />**定义工作流块/节点**|Defined by function decorator `@workflow.chunk()`<br />Defined chunks will be found in dict `workflow.chunks`<br />Default named by function name.<br /><br />由装饰器`@workflow.chunk()`装饰执行函数完成定义<br />定义完成的工作块将可以在字典`workflow.chunks`中找到<br />工作块默认命名与执行函数命名一致|Define node function directly<br />Register as a node by `workflow.add_node("node name", node_func)`<br /><br />直接定义函数<br />通过`workflow.add_node("node name", node_func)`方法注册成为一个node节点|
|**Start and End Points**<br /><br />**开始和结束**|Provide standard chunk type `Start` and `END`<br />Those two standard chunks will be put into `workflow.chunks` by default when workflow is created<br /><br />提供标准的chunk_type，`Start`和`End`<br />两个名为"start"和"end"的标准工作块在工作流创建时会默认内置到工作块池中|Using `workflow.set_entry_ponit("node name", node_func)` to set the node to start<br />Using `langgraph.graph.END` as the ending node<br /><br />通过`workflow.set_entry_ponit("node name", node_func)`的方式确定起点块<br />使用内置的langgraph.graph.END作为终点块|
|**Normal Connection**<br /><br />**普通连接**|Connect two chunks using`workflow.chunks["from_chunk_name"].connect_to(workflow.chunks["to_chunk_name"])`<br />Simplify expression: `.connect_to("to_chunk_name")`<br />Chain expression like `.connect_to("chunk_a").connect_to("chunk_b")` supported.<br /><br />使用`workflow.chunks["from_chunk_name"].connect_to(workflow.chunks["to_chunk_name"])`连接<br />支持使用`.connect_to("to_chunk_name")`方式简写<br />支持`.connect_to("chunk_a").connect_to("chunk_b")`这样的链式表达|Connect two nodes using `workflow.add_edge("from_node_name", "to_node_name")`<br /><br />使用`workflow.add_edge("from_node_name", "to_node_name")`连接两个node节点|
|**Conditional Connection**<br /><br />**条件连接**|Using expression like `.if_condition(lambda value, storage: value==1)` in chain expression<br />Get return value from last chunk as `value` and get workflow storage as `storage`<br />Support `.if_condition()`, `.else_condition()` currently<br />Can not support more than 2 conditions branches right now or use more "if-else" combination to express but we're working on it<br /><br />使用类似`.if_condition(lambda value, storage: value==1)`的形式进行条件判断表达<br />从上一个连接块获取函数return的返回值作为`value`，使用工作流全局存储作为`storage`<br />目前只能支持不超过两种情况的分支，或是使用多个if-else组合来进行表达，我们正在改进这一点|Using `workflow.add_conditional_edges("from_node_name", condition_func, { "return_value_1": "to_node_name_1", ... })` to define conditional edges<br />Support return more than 2 conditions to branch<br /><br />使用`workflow.add_conditional_edges("from_node_name", condition_func, { "return_value_1": "to_node_name_1", ... })`进行表达<br />支持在一个条件判断函数返回2个以上的条件结果，并进行分支规划|
|**Loop by List**<br />**基于列表的循环**|Using `.loop_with(sub_workflow)` to pass items in list to sub workflow one by one and get a list return result<br /><br />支持使用`.loop_with(sub_workflow)`将上游块的列表中的元素一个一个地传递给子工作流sub workflow，并从子工作流获得汇集成列表的返回结果|Not found in quick start guide book.<br /><br />在快速入门的相关文档中未找到|
|**Parallel Branch**<br />并行分支|You can connect one chunk to two or more different chunks and connect all these chunks to different handles of the end chunk to make a parallel branch<br /><br />你可以通过将一个工作块连接多个下游块，并将这些下游块连接到终点块的不同端点上（不同端点才会等待上游全部任务的完成）来构造并行分支|Can not support, if you try to connect one node to two or more different nodes you will receive an error<br /><br />不支持，如果你尝试将一个node下游同时连接多个node，你会收到一个错误警告|


## Code Compatation 对比

### LangGraph

因为不能使用平行分支，所以将平行部分注释掉了

In [ ]:
from typing import TypedDict, Optional, Annotated
from langgraph.graph import StateGraph, END

class State(TypedDict):
  def __init__(self, schema)->None:
    self.support_multiple_edges = True

  times: Optional[int] = None
  text: Optional[str] = None

workflow = StateGraph(State)

def start(state):
  return

def add_times(state):
  return { "times": state.get("times") + 1 }

def duplicate_text(state):
  text = state.get("text")
  text = f"{ text } { text }"
  return { "text": text }

def say_hi(state):
  print(state.get("times"), "hi")
  print(state.get("text"))

workflow.add_node("start", start)
workflow.add_node("add_times", add_times)
workflow.add_node("say_hi", say_hi)

workflow.set_entry_point("start")
workflow.add_edge("start", "add_times")
#workflow.add_edge("start", "duplicate_text")
#workflow.add_edge("duplicate_text", "say_hi")
workflow.add_edge("say_hi", "add_times")

workflow.add_conditional_edges(
  "add_times",
  lambda state: "end" if state.get("times") >= 3 else "continue",
  {
    "end": END,
    "continue": "say_hi"
  }
)

app = workflow.compile()

# Draw Mermaid Graph Code
#print(app.get_graph().draw_mermaid())

app.invoke({ "times": 0, "text": "Agently" })

### Agently Workflow

In [ ]:
import Agently

workflow = Agently.Workflow()

@workflow.chunk()
def _start(inputs, storage):
  return

@workflow.chunk()
def add_times(inputs, storage):
  times = storage.get("times")
  times += 1
  storage.set("times", times)

@workflow.chunk()
def duplicate_text(inputs, storage):
  text = storage.get("text")
  storage.set("text", f"{ text } { text }")

@workflow.chunk()
def say_hi(inputs, storage):
  print(storage.get("times"), "hi")
  print(storage.get("text"))

(
  workflow
    .connect_to("_start")
    .connect_to("add_times")
    .connect_to("say_hi.handle_1")
      .if_condition(lambda value, storage: storage.get("times") >= 3)
        .connect_to("end")
      .else_condition()
        .connect_to("_start")
)
(
  workflow.chunks["_start"]
    .connect_to("duplicate_text")
    .connect_to("say_hi.handle_2")
)

# Draw Mermaid Graph Code
#print(workflow.draw())
workflow.start(storage = { "times": 0, "text": "Agently" })

---

[**_<font color = "red">Agent</font><font color = "blue">ly</font>_** Framework - Speed up your AI application development](https://github.com/Maplemx/Agently)